In [1]:
import torch
import torch.nn as nn
import torch.onnx
import torchvision

from quantize_utils import static_quantize_model
import hyperparams as hparams
from model import load_model

MODEL_PATH = "/home/namdng/garbage_classifier/models/resnet50_tuned_lr_1e-3_bs_64_sche-f0.2-p6/ckpt_57_0.9597.pth"
BATCH_SIZE = 4

device = "cpu"

### Load model
class_names = ["cardboard_paper", "glass", "metal", "others", "plastic"]
model_conv = load_model(hparams.BACKBONE, hparams.NUM_IMMEDIATE_FEATURES, len(class_names), hparams.DROPOUT_RATE)
if "quantize" in MODEL_PATH:
    device = "cpu"
    model_conv = static_quantize_model(model_conv, dataloader=None, backend="fbgemm")

print("Loading model: ", MODEL_PATH)
model_conv.load_state_dict(torch.load(MODEL_PATH, weights_only=True))
model_conv = model_conv.to(device)
model_conv.eval()

# Input to the model
x = torch.randn(BATCH_SIZE, 3, hparams.IMAGE_SIZE, hparams.IMAGE_SIZE, device=device)
torch_out = model_conv(x)
print(torch_out)



Loading model:  /home/namdng/garbage_classifier/models/resnet50_tuned_lr_1e-3_bs_64_sche-f0.2-p6/ckpt_57_0.9597.pth
tensor([[ -0.6570,   0.2405,  -9.7473,   4.9803,  -2.1334],
        [ -2.1151,   0.4455,  -9.6111,   5.8562,  -1.5623],
        [ -0.1978,   0.0618, -10.4337,   4.8975,  -2.3741],
        [ -1.3153,   0.1393, -10.4668,   6.0580,  -2.0621]],
       grad_fn=<AddmmBackward0>)


### TorchScript-based ONNX Exporter

In [2]:
# Export the model
torch.onnx.export(model_conv,                # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "gc_torchscript.onnx",                 # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

### TorchDynamo-based ONNX Exporter

In [ ]:
onnx_program = torch.onnx.export(model_conv, x, dynamo=True)
onnx_program.save("gc_torchdynamo.onnx")

In [ ]:
import onnxruntime
import numpy as np

ort_session = onnxruntime.InferenceSession("gc_torchscript.onnx", providers=["CPUExecutionProvider"])

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

print(to_numpy(torch_out), ort_outs[0])

# compare ONNX Runtime and PyTorch results
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("Exported model has been tested with ONNXRuntime, and the result looks good!")

### convert test images to RGB

In [ ]:
import os
from PIL import Image

DATA_DIR = "data_split/val"

for class_name in os.listdir(DATA_DIR):
    class_dir = os.path.join(DATA_DIR, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = Image.open(image_path)
        image_mode = image.mode
        if image_mode != "RGB":
            image = image.convert("RGB")
            image.save(image_path)